# Disable contracts

In [2]:
import os

os.environ["DISABLE_CONTRACTS"] = "true"

# Importing necessary libraries

In [124]:
import numpy as np
from numpy.polynomial.polynomial import polyval

from scipy.special import legendre

In [211]:
from osol.extremum.cybernetics.models.fmu_model import FMUModel
from osol.extremum.cybernetics.controllers_and_tools.saturation_limiter import SaturationLimiter

from osol.extremum.optimization.basic.terminator import MaxTimeTerminator

from osol.extremum.optimization.algorithms.statistical_anti_gradient_random_search import StatisticalAntiGradientRandomSearch
from osol.extremum.optimization.algorithms.big_bang_big_crunch import BigBangBigCrunch

In [5]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

# Cybernetics block

In [6]:
model = FMUModel(
    model="satellite_stabilization/satellite.fmu",
    description="satellite_stabilization/satellite.json",
    needs_compilation=True)

Compiling Satellite_Stabilization.dylib...
/var/folders/d_/2kzh_1ks41d7tgrmt7fnp8pc0000gn/T/tmpo98663h_/sources
gcc -c -I. -I/Users/wol4aravio/.local/share/virtualenvs/Almaz_Report_2019-mIQM5fFf/lib/python3.6/site-packages/fmpy/c-code/fmi2 -DCO_SIMULATION -DFMI2_FUNCTION_PREFIX=Satellite_Stabilization_ Satellite_Stabilization.c /Users/wol4aravio/.local/share/virtualenvs/Almaz_Report_2019-mIQM5fFf/lib/python3.6/site-packages/fmpy/c-code/fmi2/fmi2_wrapper.c && gcc -shared -oSatellite_Stabilization.dylib *.o


In [7]:
initial_state = {
    "p0": 24.0,
    "q0": 16.0,
    "r0": 16.0
}

termination_time = 1.0
time_step=1e-2

In [8]:
max_abs_control = 500.0
limiter = SaturationLimiter(min_value=-max_abs_control, max_value=max_abs_control)

# Auxiliary functions

In [9]:
def run_simulations(controllers):
    return model.simulate(
        initial_state=initial_state,
        termination_time=termination_time,
        dt=time_step,
        controllers=controllers)

In [83]:
def criterion(states, control_history, tolerance=1e-2, penalty=1e3, Lp=2):
    integral_criterion_value = 0.0
    for dt, u1, u2, u3 in zip(states["t"][1:] - states["t"][:-1], control_history["u1"], control_history["u2"], control_history["u3"]):
        integral_criterion_value += dt * (np.abs(u1) + np.abs(u2) + np.abs(u3))
    
    terminal_criterion_value = 0.0
    terminal_states = states["p"][-1], states["q"][-1], states["r"][-1]
    for v in terminal_states:
        v_ = np.abs(v)
        if v_ >= tolerance:
            terminal_criterion_value += np.power(penalty * v_, Lp)
        else:
            terminal_criterion_value += v_
    
    return integral_criterion_value, terminal_criterion_value, integral_criterion_value + terminal_criterion_value

In [53]:
def draw_controls(states, controls):
    trace_u1 = go.Scatter(
        x = states["t"][:-1],
        y = controls["u1"],
        mode = "lines",
        name = "u1")

    trace_u2 = go.Scatter(
        x = states["t"][:-1],
        y = controls["u2"],
        mode = "lines",
        name = "u2")

    trace_u3 = go.Scatter(
        x = states["t"][:-1],
        y = controls["u3"],
        mode = "lines",
        name = "u3")

    data = [trace_u1, trace_u2, trace_u3]

    iplot(data)

In [54]:
def draw_states(states, controls):
    trace_p = go.Scatter(
        x = states["t"],
        y = states["p"],
        mode = "lines",
        name = "p")

    trace_q = go.Scatter(
        x = states["t"],
        y = states["q"],
        mode = "lines",
        name = "q")

    trace_r = go.Scatter(
        x = states["t"],
        y = states["r"],
        mode = "lines",
        name = "r")

    data = [trace_p, trace_q, trace_r]

    iplot(data)

# Checking poly controllers

In [55]:
def create_poly_controllers(v):
    return {
        "u1": {
            "f": lambda t: polyval(t, v[0::3]),
            "inputs": ["t"]
        },
        "u2": {
            "f": lambda t: polyval(t, v[1::3]),
            "inputs": ["t"]
        },
        "u3": {
            "f": lambda t: polyval(t, v[2::3]),
            "inputs": ["t"]
        }
    }

In [56]:
s, c = run_simulations(create_poly_controllers(np.random.uniform(-100.0, 100.0, size=6)))

In [57]:
draw_controls(s, c)

In [58]:
draw_states(s, c)

# Finding best linear controller

In [87]:
search_area = {v_name: (-500, 500) for v_name in ["a1", "a2", "a3", "b1", "b2", "b3"]}

# Selecting algorithm configuration

In [88]:
algorithm = BigBangBigCrunch(
    number_of_points=25, 
    scatter_parameter=1e2, 
    quality_mode=lambda v, cb, ob: v - ob + 1e-7)

In [89]:
f = MaxTimeTerminator(
    lambda v: criterion(*run_simulations(controllers=create_poly_controllers(v.to_numpy_array())))[2],
    mode="dummy",
    max_time="m:5")

In [90]:
best_linear_controller = algorithm.optimize(f, search_area)

In [91]:
s, c = run_simulations(create_poly_controllers(best_linear_controller.to_numpy_array()))

In [92]:
s["p"][-1], s["q"][-1], s["r"][-1]

(0.005688806999224822, -0.004391418933557517, -0.004513408025381974)

In [96]:
draw_controls(s, c)

In [97]:
draw_states(s, c)

In [99]:
criterion(s, c)

(327.62866578369164, 0.014593633958164313, 327.6432594176498)

# Finding best quadratic controller

In [101]:
search_area = {v_name: (-500, 500) for v_name in ["a1", "a2", "a3", "b1", "b2", "b3", "с1", "с2", "с3"]}

# Selecting algorithm configuration

In [110]:
algorithm = BigBangBigCrunch(
    number_of_points=25, 
    scatter_parameter=1e2, 
    quality_mode=lambda v, cb, ob: v - ob + 1e-7)

In [111]:
f = MaxTimeTerminator(
    lambda v: criterion(*run_simulations(controllers=create_poly_controllers(v.to_numpy_array())))[2],
    mode="dummy",
    max_time="m:10")

In [112]:
best_quadratic_controller = algorithm.optimize(f, search_area)

In [113]:
s, c = run_simulations(create_poly_controllers(best_quadratic_controller.to_numpy_array()))

In [114]:
s["p"][-1], s["q"][-1], s["r"][-1]

(-0.0073276211994605725, 0.0030821619501079853, 0.0035404235828691774)

In [115]:
draw_controls(s, c)

In [116]:
draw_states(s, c)

In [117]:
criterion(s, c)

(279.7279669637969, 0.013950206732437735, 279.74191717052935)

# Finding best controller via basis decomposition

In [241]:
dim_basis = 3

search_area = dict()
for i in [1, 2, 3]:
    for j in range(1, dim_basis + 1):
        search_area[f"c{i}_{j}"] = (-500.0, 500.0)

In [242]:
def get_decomposition(t, coeff, scale=500):
    tau = 2.0 * (t - 0.5)
    poly_values = [legendre(i)(tau) for i in range(len(coeff))]
    return np.sum(poly_values * coeff)

In [243]:
def create_legendre_controllers(v):
    return {
        "u1": {
            "f": lambda t: get_decomposition(t, coeff=v[0::3]),
            "inputs": ["t"]
        },
        "u2": {
            "f": lambda t: get_decomposition(t, coeff=v[1::3]),
            "inputs": ["t"]
        },
        "u3": {
            "f": lambda t: get_decomposition(t, coeff=v[2::3]),
            "inputs": ["t"]
        }
    }

In [244]:
algorithm = StatisticalAntiGradientRandomSearch(
    radius=2.5, 
    number_of_samples=10)

In [245]:
f = MaxTimeTerminator(
    lambda v: criterion(*run_simulations(controllers=create_legendre_controllers(v.to_numpy_array())))[2],
    mode="dummy",
    max_time="h:1")

In [ ]:
best_legendre_controller = algorithm.optimize(f, search_area)

In [ ]:
s, c = run_simulations(create_legendre_controllers(best_legendre_controller.to_numpy_array()))

In [ ]:
s["p"][-1], s["q"][-1], s["r"][-1]

In [ ]:
draw_controls(s, c)

In [ ]:
draw_states(s, c)

In [ ]:
criterion(s, c)

# Purging temporary files

In [ ]:
model._purge()